In [1]:
import xarray as xr
from FINE import utils
import FINE as fn
import numpy as np
import pandas as pd
import os
import warnings

try:
    import FINE.spagat.manager as spm
    import FINE.spagat.representation as spr
except ImportError:
    warnings.warn('The Spagat python package could not be imported.')

import geopandas as gpd

## dataset - multi_node_test_esM_init

In [2]:
def getData():
    inputDataPath = r'C:\Users\s.patil\Documents\code\fine\examples\Multi-regional_Energy_System_Workflow\InputData'
    data = {}

    # Onshore data
    capacityMax = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'Wind', 'maxCapacityOnshore_GW_el.xlsx'),
                                index_col=0, squeeze=True)
    operationRateMax = pd.read_excel(
        os.path.join(inputDataPath, 'SpatialData', 'Wind', 'maxOperationRateOnshore_el.xlsx'),
                                                  header = 0, index_col = 0)

    data.update({'Wind (onshore), capacityMax': capacityMax})
    data.update({'Wind (onshore), operationRateMax': operationRateMax})

    # Offshore data
    capacityMax = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'Wind', 'maxCapacityOffshore_GW_el.xlsx'),
                                index_col=0, squeeze=True)
    operationRateMax = pd.read_excel(
        os.path.join(inputDataPath, 'SpatialData', 'Wind', 'maxOperationRateOffshore_el.xlsx'),
                                                  header = 0, index_col = 0)

    data.update({'Wind (offshore), capacityMax': capacityMax})
    data.update({'Wind (offshore), operationRateMax': operationRateMax})

    # PV data
    capacityMax = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'PV', 'maxCapacityPV_GW_el.xlsx'),
                                index_col=0, squeeze=True)
    operationRateMax = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'PV', 'maxOperationRatePV_el.xlsx'),
                                                  header = 0, index_col = 0)

    data.update({'PV, capacityMax': capacityMax})
    data.update({'PV, operationRateMax': operationRateMax})

    # Run of river data
    capacityFix = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'HydroPower', 'fixCapacityROR_GW_el.xlsx'),
                                index_col=0, squeeze=True)
    operationRateFix = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'HydroPower',
                                                  'fixOperationRateROR.xlsx'),
                                                  header = 0, index_col = 0)

    data.update({'Existing run-of-river plants, capacityFix': capacityFix})
    data.update({'Existing run-of-river plants, operationRateFix': operationRateFix})

    # Biogas data
    operationRateMax = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'Biogas',
                                                  'biogasPotential_GWh_biogas.xlsx'),
                                                  header = 0, index_col = 0)

    data.update({'Biogas, operationRateMax': operationRateMax})

    biogasCommodityCostTimeSeries = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'Biogas',
                                                  'biogasPriceTimeSeries_MrdEuro_GWh.xlsx'),
                                                  header = 0, index_col = 0)

    data.update({'Biogas, commodityCostTimeSeries': biogasCommodityCostTimeSeries})

    # Natural gas data
    naturalGasCommodityCostTimeSeries = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'NaturalGas',
                                                  'naturalGasPriceTimeSeries_MrdEuro_GWh.xlsx'),
                                                  header = 0, index_col = 0)

    data.update({'Natural Gas, commodityCostTimeSeries': naturalGasCommodityCostTimeSeries})

    # Natural gas plant data
    capacityMax = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'NaturalGasPlants',
                                             'existingCombinedCycleGasTurbinePlantsCapacity_GW_el.xlsx'),
                                index_col=0, squeeze=True)

    data.update({'Existing CCGT plants (methane), capacityMax': capacityMax})

    # Hydrogen salt cavern data
    capacityMax = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'GeologicalStorage',
                                             'existingSaltCavernsCapacity_GWh_methane.xlsx'),
                                index_col=0, squeeze=True) * 3 / 10

    data.update({'Salt caverns (hydrogen), capacityMax': capacityMax})

    # Methane salt cavern data
    capacityMax = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'GeologicalStorage',
                                             'existingSaltCavernsCapacity_GWh_methane.xlsx'),
                                index_col=0, squeeze=True)

    data.update({'Salt caverns (methane), capacityMax': capacityMax})

    # Pumped hydro storage data
    capacityFix = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'HydroPower',
                                             'fixCapacityPHS_storage_GWh_energyPHS.xlsx'),
                                index_col=0, squeeze=True)

    data.update({'Pumped hydro storage, capacityFix': capacityFix})

    # AC cables data
    capacityFix = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'ElectricGrid',
                                             'ACcableExistingCapacity_GW_el.xlsx'),
                                index_col=0, header=0)

    data.update({'AC cables, capacityFix': capacityFix})

    reactances = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'ElectricGrid',
                                            'ACcableReactance.xlsx'),
                                index_col=0, header=0)

    data.update({'AC cables, reactances': reactances})

    # DC cables data
    capacityFix = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'ElectricGrid',
                                             'DCcableExistingCapacity_GW_el.xlsx'),
                                index_col=0, header=0)
    distances = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'ElectricGrid',
                                           'DCcableLength_km.xlsx'),
                              index_col=0, header=0)
    losses = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'ElectricGrid',
                                        'DCcableLosses.xlsx'),
                           index_col=0, header=0)

    data.update({'DC cables, capacityFix': capacityFix})
    data.update({'DC cables, distances': distances})
    data.update({'DC cables, losses': losses})

    # Pipelines data
    eligibility = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'Pipelines',
                                             'pipelineIncidence.xlsx'), index_col=0, header=0)
    distances = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'Pipelines', 'pipelineLength.xlsx'),
                              index_col=0, header=0)

    data.update({'Pipelines, eligibility': eligibility})
    data.update({'Pipelines, distances': distances})

    # Electricity demand data
    operationRateFix = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'Demands',
                                                  'electricityDemand_GWh_el.xlsx'),
                                                  header = 0, index_col = 0)

    data.update({'Electricity demand, operationRateFix': operationRateFix})

    # Hydrogen demand data
    operationRateFix = pd.read_excel(os.path.join(inputDataPath, 'SpatialData', 'Demands',
                                                  'hydrogenDemand_GWh_hydrogen.xlsx'),
                                                  header = 0, index_col = 0)

    data.update({'Hydrogen demand, operationRateFix': operationRateFix})

    return data

In [3]:
data = getData()

In [4]:
# 2. Create an energy system model instance
locations = {'cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7'}
commodityUnitDict = {'electricity': r'GW$_{el}$', 'methane': r'GW$_{CH_{4},LHV}$', 'biogas': r'GW$_{biogas,LHV}$',
                         'CO2': r'Mio. t$_{CO_2}$/h', 'hydrogen': r'GW$_{H_{2},LHV}$'}
commodities = {'electricity', 'hydrogen', 'methane', 'biogas', 'CO2'}
numberOfTimeSteps=8760
hoursPerTimeStep=1

esM = fn.EnergySystemModel(locations=locations, commodities=commodities, numberOfTimeSteps=8760,
                               commodityUnitsDict=commodityUnitDict,
                               hoursPerTimeStep=1, costUnit='1e9 Euro', lengthUnit='km', verboseLogLevel=0)

CO2_reductionTarget = 1


# 3. Add commodity sources to the energy system model
## 3.1. Electricity sources
### Wind onshore

esM.add(fn.Source(esM=esM, name='Wind (onshore)', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['Wind (onshore), operationRateMax'],
                  capacityMax=data['Wind (onshore), capacityMax'],
                  investPerCapacity=1.1, opexPerCapacity=1.1*0.02, interestRate=0.08,
                  economicLifetime=20))

data['Wind (onshore), operationRateMax'].sum()


### Wind offshore

esM.add(fn.Source(esM=esM, name='Wind (offshore)', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['Wind (offshore), operationRateMax'],
                  capacityMax=data['Wind (offshore), capacityMax'],
                  investPerCapacity=2.3, opexPerCapacity=2.3*0.02, interestRate=0.08,
                  economicLifetime=20))

data['Wind (offshore), operationRateMax'].sum()

### PV

esM.add(fn.Source(esM=esM, name='PV', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['PV, operationRateMax'], capacityMax=data['PV, capacityMax'],
                  investPerCapacity=0.65, opexPerCapacity=0.65*0.02, interestRate=0.08,
                  economicLifetime=25))

data['PV, operationRateMax'].sum()

### Exisisting run-of-river hydroelectricity plants

esM.add(fn.Source(esM=esM, name='Existing run-of-river plants', commodity='electricity',
                  hasCapacityVariable=True,
                  operationRateFix=data['Existing run-of-river plants, operationRateFix'], tsaWeight=0.01,
                  capacityFix=data['Existing run-of-river plants, capacityFix'],
                  investPerCapacity=0, opexPerCapacity=0.208))

## 3.2. Methane (natural gas and biogas)
### Natural gas
esM.add(fn.Source(esM=esM, name='Natural gas purchase', commodity='methane',
                  hasCapacityVariable=False, commodityCostTimeSeries=data['Natural Gas, commodityCostTimeSeries']))

### Biogas
esM.add(fn.Source(esM=esM, name='Biogas purchase', commodity='biogas',
                  operationRateMax=data['Biogas, operationRateMax'], hasCapacityVariable=False,
                  commodityCostTimeSeries=data['Biogas, commodityCostTimeSeries']))


## 3.3 CO2
### CO2

esM.add(fn.Source(esM=esM, name='CO2 from enviroment', commodity='CO2',
                  hasCapacityVariable=False, commodityLimitID='CO2 limit', yearlyLimit=366*(1-CO2_reductionTarget)))


# 4. Add conversion components to the energy system model

### Combined cycle gas turbine plants

esM.add(fn.Conversion(esM=esM, name='CCGT plants (methane)', physicalUnit=r'GW$_{el}$',
                      commodityConversionFactors={'electricity':1, 'methane':-1/0.625, 'CO2':201*1e-6/0.625},
                      hasCapacityVariable=True,
                      investPerCapacity=0.65, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))


### New combined cycle gas turbine plants for biogas

esM.add(fn.Conversion(esM=esM, name='New CCGT plants (biogas)', physicalUnit=r'GW$_{el}$',
                      commodityConversionFactors={'electricity':1, 'biogas':-1/0.635},
                      hasCapacityVariable=True,
                      investPerCapacity=0.7, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))


### New combined cycly gas turbines for hydrogen

esM.add(fn.Conversion(esM=esM, name='New CCGT plants (hydrogen)', physicalUnit=r'GW$_{el}$',
                      commodityConversionFactors={'electricity':1, 'hydrogen':-1/0.6},
                      hasCapacityVariable=True,
                      investPerCapacity=0.7, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))

### Electrolyzers

esM.add(fn.Conversion(esM=esM, name='Electroylzers', physicalUnit=r'GW$_{el}$',
                      commodityConversionFactors={'electricity':-1, 'hydrogen':0.7},
                      hasCapacityVariable=True,
                      investPerCapacity=0.5, opexPerCapacity=0.5*0.025, interestRate=0.08,
                      economicLifetime=10))

### rSOC

capexRSOC=1.5

esM.add(fn.Conversion(esM=esM, name='rSOEC', physicalUnit=r'GW$_{el}$', linkedConversionCapacityID='rSOC',
                      commodityConversionFactors={'electricity':-1, 'hydrogen':0.6},
                      hasCapacityVariable=True,
                      investPerCapacity=capexRSOC/2, opexPerCapacity=capexRSOC*0.02/2, interestRate=0.08,
                      economicLifetime=10))

esM.add(fn.Conversion(esM=esM, name='rSOFC', physicalUnit=r'GW$_{el}$', linkedConversionCapacityID='rSOC',
                      commodityConversionFactors={'electricity':1, 'hydrogen':-1/0.6},
                      hasCapacityVariable=True,
                      investPerCapacity=capexRSOC/2, opexPerCapacity=capexRSOC*0.02/2, interestRate=0.08,
                      economicLifetime=10))


# 5. Add commodity storages to the energy system model
## 5.1. Electricity storage
### Lithium ion batteries

esM.add(fn.Storage(esM=esM, name='Li-ion batteries', commodity='electricity',
                   hasCapacityVariable=True, chargeEfficiency=0.95,
                   cyclicLifetime=10000, dischargeEfficiency=0.95, selfDischarge=1-(1-0.03)**(1/(30*24)),
                   chargeRate=1, dischargeRate=1, doPreciseTsaModeling=False,
                   investPerCapacity=0.151, opexPerCapacity=0.002, interestRate=0.08,
                   economicLifetime=22))

## 5.2. Hydrogen storage
### Hydrogen filled salt caverns

esM.add(fn.Storage(esM=esM, name='Salt caverns (hydrogen)', commodity='hydrogen',
                   hasCapacityVariable=True, capacityVariableDomain='continuous',
                   capacityPerPlantUnit=133,
                   chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
                   stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=data['Salt caverns (hydrogen), capacityMax'],
                   investPerCapacity=0.00011, opexPerCapacity=0.00057, interestRate=0.08,
                   economicLifetime=30))


## 5.3. Methane storage
### Methane filled salt caverns

esM.add(fn.Storage(esM=esM, name='Salt caverns (biogas)', commodity='biogas',
                   hasCapacityVariable=True, capacityVariableDomain='continuous',
                   capacityPerPlantUnit=443,
                   chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
                   stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=data['Salt caverns (methane), capacityMax'],
                   investPerCapacity=0.00004, opexPerCapacity=0.00001, interestRate=0.08,
                   economicLifetime=30))


## 5.4 Pumped hydro storage
### Pumped hydro storage

esM.add(fn.Storage(esM=esM, name='Pumped hydro storage', commodity='electricity',
                   chargeEfficiency=0.88, dischargeEfficiency=0.88,
                   hasCapacityVariable=True, selfDischarge=1-(1-0.00375)**(1/(30*24)),
                   chargeRate=0.16, dischargeRate=0.12, capacityFix=data['Pumped hydro storage, capacityFix'],
                   investPerCapacity=0, opexPerCapacity=0.000153))


# 6. Add commodity transmission components to the energy system model
## 6.1. Electricity transmission
### AC cables

esM.add(fn.LinearOptimalPowerFlow(esM=esM, name='AC cables', commodity='electricity',
                                  hasCapacityVariable=True, capacityFix=data['AC cables, capacityFix'],
                                  reactances=data['AC cables, reactances']))

### DC cables

esM.add(fn.Transmission(esM=esM, name='DC cables', commodity='electricity', losses=data['DC cables, losses'],
                        distances=data['DC cables, distances'],
                        hasCapacityVariable=True, capacityFix=data['DC cables, capacityFix']))


## 6.2 Methane transmission
### Methane pipeline

esM.add(fn.Transmission(esM=esM, name='Pipelines (biogas)', commodity='biogas',
                        distances=data['Pipelines, distances'],
                        hasCapacityVariable=True, hasIsBuiltBinaryVariable=False, bigM=300,
                            capacityMax=data['Pipelines, eligibility']*15, sharedPotentialID='pipelines',
                            investPerCapacity=0.000037, investIfBuilt=0.000314,
                            interestRate=0.08, economicLifetime=40))

## 6.3 Hydrogen transmission
### Hydrogen pipelines

esM.add(fn.Transmission(esM=esM, name='Pipelines (hydrogen)', commodity='hydrogen',
                            distances=data['Pipelines, distances'],
                            hasCapacityVariable=True, hasIsBuiltBinaryVariable=False, bigM=300,
                            locationalEligibility=data['Pipelines, eligibility'],
                            capacityMax=data['Pipelines, eligibility']*15, sharedPotentialID='pipelines',
                            investPerCapacity=0.000177, investIfBuilt=0.00033,
                            interestRate=0.08, economicLifetime=40))

# 7. Add commodity sinks to the energy system model
## 7.1. Electricity sinks
### Electricity demand

esM.add(fn.Sink(esM=esM, name='Electricity demand', commodity='electricity',
                    hasCapacityVariable=False, operationRateFix=data['Electricity demand, operationRateFix']))

## 7.2. Hydrogen sinks
### Fuel cell electric vehicle (FCEV) demand

FCEV_penetration=0.5
esM.add(fn.Sink(esM=esM, name='Hydrogen demand', commodity='hydrogen', hasCapacityVariable=False,
                    operationRateFix=data['Hydrogen demand, operationRateFix']*FCEV_penetration))

## 7.3. CO2 sinks
### CO2 exiting the system's boundary

esM.add(fn.Sink(esM=esM, name='CO2 to enviroment', commodity='CO2',
                    hasCapacityVariable=False, commodityLimitID='CO2 limit', yearlyLimit=366*(1-CO2_reductionTarget)))

# 8. Optimize energy system model

# esM.cluster(numberOfTypicalPeriods=3)

# esM.optimize(timeSeriesAggregation=True, solver = solver)


The distances of a component are set to a normalized value of 1.


In [5]:
multi_node_test_esM_init = esM

## esm_dict, component_dict

In [6]:
esm_dict, component_dict = fn.dictIO.exportToDict(multi_node_test_esM_init)

In [7]:
esm_dict

{'locations': {'cluster_0',
  'cluster_1',
  'cluster_2',
  'cluster_3',
  'cluster_4',
  'cluster_5',
  'cluster_6',
  'cluster_7'},
 'commodities': {'CO2', 'biogas', 'electricity', 'hydrogen', 'methane'},
 'commodityUnitsDict': {'electricity': 'GW$_{el}$',
  'methane': 'GW$_{CH_{4},LHV}$',
  'biogas': 'GW$_{biogas,LHV}$',
  'CO2': 'Mio. t$_{CO_2}$/h',
  'hydrogen': 'GW$_{H_{2},LHV}$'},
 'numberOfTimeSteps': 8760,
 'hoursPerTimeStep': 1,
 'costUnit': '1e9 Euro',
 'lengthUnit': 'km',
 'verboseLogLevel': 0}

In [8]:
component_dict

{'Source': {'Wind (onshore)': {'name': 'Wind (onshore)',
   'commodity': 'electricity',
   'hasCapacityVariable': True,
   'capacityVariableDomain': 'continuous',
   'capacityPerPlantUnit': 1,
   'hasIsBuiltBinaryVariable': False,
   'bigM': None,
   'operationRateMax': None,
   'operationRateFix': None,
   'tsaWeight': 1,
   'commodityLimitID': None,
   'yearlyLimit': None,
   'locationalEligibility': cluster_0    1.0
   cluster_1    1.0
   cluster_2    1.0
   cluster_3    1.0
   cluster_4    1.0
   cluster_5    1.0
   cluster_6    1.0
   cluster_7    1.0
   Name: 0, dtype: float64,
   'capacityMin': None,
   'capacityMax': cluster_0    45.068240
   cluster_1    65.918145
   cluster_2    24.894326
   cluster_3    79.331044
   cluster_4    91.721729
   cluster_5    52.801808
   cluster_6    44.974846
   cluster_7     8.524905
   Name: 0, dtype: float64,
   'partLoadMin': None,
   'sharedPotentialID': None,
   'linkedQuantityID': None,
   'capacityFix': None,
   'isBuiltFix': None,
   '

## generate_iteration_dicts()

In [7]:
def generate_iteration_dicts(esm_dict, component_dict):
    """Creates iteration dictionaries that contain descriptions of all dataframes and series of the dictionaries esm_dict 
    and component_dict.
    
    :param esm_dict: dictionary containing information about the esM instance
    :type esm_dict: dict

    :param component_dict: dictionary containing information about the esM instance's components
    :type component_dict: dict
 
    :return: df_iteration_dict, series_iteration_dict
    """

    df_iteration_dict = {}

    for classname in component_dict:

        for component in component_dict[classname]:            

            for variable_description, data in component_dict[classname][component].items():
                description_tuple = (classname, component)

                if isinstance(data, pd.DataFrame):
                    if variable_description not in df_iteration_dict.keys():
                        df_iteration_dict[variable_description] = [description_tuple]
                    else:
                        df_iteration_dict[variable_description].append(description_tuple)

    series_iteration_dict = {}

    for classname in component_dict:

        for component in component_dict[classname]:            

            for variable_description, data in component_dict[classname][component].items():
                description_tuple = (classname, component)

                if isinstance(data, pd.Series):
                    if variable_description not in series_iteration_dict.keys():
                        series_iteration_dict[variable_description] = [description_tuple]
                    else:
                        series_iteration_dict[variable_description].append(description_tuple)

    return df_iteration_dict, series_iteration_dict

In [8]:
df_iteration_dict, series_iteration_dict = generate_iteration_dicts(esm_dict, component_dict)

In [9]:
df_iteration_dict

{'operationRateMax': [('Source', 'Wind (onshore)'),
  ('Source', 'Wind (offshore)'),
  ('Source', 'PV'),
  ('Source', 'Biogas purchase')],
 'operationRateFix': [('Source', 'Existing run-of-river plants'),
  ('Sink', 'Electricity demand'),
  ('Sink', 'Hydrogen demand')],
 'commodityCostTimeSeries': [('Source', 'Natural gas purchase'),
  ('Source', 'Biogas purchase')]}

In [10]:
series_iteration_dict

{'locationalEligibility': [('Source', 'Wind (onshore)'),
  ('Source', 'Wind (offshore)'),
  ('Source', 'PV'),
  ('Source', 'Existing run-of-river plants'),
  ('Source', 'Natural gas purchase'),
  ('Source', 'Biogas purchase'),
  ('Source', 'CO2 from enviroment'),
  ('Sink', 'Electricity demand'),
  ('Sink', 'Hydrogen demand'),
  ('Sink', 'CO2 to enviroment'),
  ('Conversion', 'CCGT plants (methane)'),
  ('Conversion', 'New CCGT plants (biogas)'),
  ('Conversion', 'New CCGT plants (hydrogen)'),
  ('Conversion', 'Electroylzers'),
  ('Conversion', 'rSOEC'),
  ('Conversion', 'rSOFC'),
  ('Storage', 'Li-ion batteries'),
  ('Storage', 'Salt caverns (hydrogen)'),
  ('Storage', 'Salt caverns (biogas)'),
  ('Storage', 'Pumped hydro storage'),
  ('LinearOptimalPowerFlow', 'AC cables'),
  ('Transmission', 'DC cables'),
  ('Transmission', 'Pipelines (biogas)'),
  ('Transmission', 'Pipelines (hydrogen)')],
 'capacityMax': [('Source', 'Wind (onshore)'),
  ('Source', 'Wind (offshore)'),
  ('Source', 

## dimensional_data_to_xarray_dataset()

In [13]:
def dimensional_data_to_xarray_dataset(esm_dict, component_dict):
    """Outputs all dimensional data to an xarray dataset.
    
    Note:  Here, "dimensional data" refers to data containing at least one of the dimensions of time and space.

    :param esm_dict: dictionary containing information about the esM instance
    :type esm_dict: dict

    :param component_dict: dictionary containing information about the esM instance's components
    :type component_dict: dict
 
    :return: ds - xarray dataset containing all dimensional data of an esM instance
    """

    locations = list(esm_dict['locations'])

    df_iteration_dict, series_iteration_dict = generate_iteration_dicts(esm_dict, component_dict)

    # iterate over iteration dicts
    ds = xr.Dataset()

    # get all regional time series (space, time)
    for variable_description, description_tuple_list in df_iteration_dict.items():
        df_dict = {} # fn.dictionary of multiindex data frames that all contain all data for one variable
        for description_tuple in description_tuple_list:
            classname, component = description_tuple

            df_description = f"{classname}, {component}"

            data = component_dict[classname][component][variable_description]

            if isinstance(data, pd.DataFrame):
                # print('data is pd.DataFrame')
                multi_index_dataframe = data.stack()
                if classname == 'LinearOptimalPowerFlow':
                    multi_index_dataframe.index.set_names("space", level=1, inplace=True)
                else:
                    multi_index_dataframe.index.set_names("space", level=2, inplace=True)
                    

                df_dict[df_description] = multi_index_dataframe # append or concat or so
                                        
        df_variable = pd.concat(df_dict)
        df_variable.index.set_names("component", level=0, inplace=True) # ?

        ds_component = xr.Dataset()
        ds_component[variable_description] = df_variable.sort_index().to_xarray()

        ds = xr.merge([ds, ds_component])

    # get all 2d data (space, space)
    for variable_description, description_tuple_list in series_iteration_dict.items():
        df_dict = {} # dictionary of multiindex data frames that all contain all data for one variable
    
        for description_tuple in description_tuple_list:
            classname, component = description_tuple

            df_description = f"{classname}, {component}"

            data = component_dict[classname][component][variable_description]

            if isinstance(data, pd.Series):

                if classname in ['Transmission', 'LinearOptimalPowerFlow']:
                    # TODO: which one of transmission's components are 2d and which 1d or dimensionless

                    df = utils.transform1dSeriesto2dDataFrame(data, locations)
                    multi_index_dataframe = df.stack()
                    multi_index_dataframe.index.set_names(["space", "space_2"], inplace=True)

                    df_dict[df_description] = multi_index_dataframe

        if len(df_dict) > 0:
            df_variable = pd.concat(df_dict)
            df_variable.index.set_names("component", level=0, inplace=True) # ?

            ds_component = xr.Dataset()
            ds_component[f"2d_{variable_description}"] = df_variable.sort_index().to_xarray()

            ds = xr.merge([ds, ds_component])

    # get all 1d data (space)
    for variable_description, description_tuple_list in series_iteration_dict.items():

        df_dict = {} # dictionary of multiindex data frames that all contain all data for one variable
        for description_tuple in description_tuple_list:
            classname, component = description_tuple
    
            df_description = f"{classname}, {component}"

            data = component_dict[classname][component][variable_description]

            if isinstance(data, pd.Series): # TODO: remove this line, as all data should be series (?)
                # if classname not in ['Transmission', 'LinearOptimalPowerFlow'] and len(data>= len(locations)):
                if classname not in ['Transmission', 'LinearOptimalPowerFlow']:
                    if len(data) >= len(locations): # TODO: this is a bugfix to remove '1d_locationalEligibility', do this properly
                        df_dict[df_description] = data.rename_axis("space")
            

        if len(df_dict) > 0:
            df_variable = pd.concat(df_dict)
            df_variable.index.set_names("component", level=0, inplace=True) # ?

            ds_component = xr.Dataset()
            ds_component[f"1d_{variable_description}"] = df_variable.sort_index().to_xarray()

            ds = xr.merge([ds, ds_component])

    return ds

In [14]:
dimensional_data_to_xarray_dataset(esm_dict, component_dict)

<xarray.Dataset>
Dimensions:                       (Period: 1, TimeStep: 8760, component: 24, space: 8, space_2: 8)
Coordinates:
  * component                     (component) object 'Conversion, CCGT plants...
  * Period                        (Period) int64 0
  * TimeStep                      (TimeStep) int64 0 1 2 3 ... 8757 8758 8759
  * space                         (space) object 'cluster_0' ... 'cluster_7'
  * space_2                       (space_2) object 'cluster_0' ... 'cluster_7'
Data variables:
    operationRateMax              (component, Period, TimeStep, space) float64 ...
    operationRateFix              (component, Period, TimeStep, space) float64 ...
    commodityCostTimeSeries       (component, Period, TimeStep, space) float64 ...
    2d_locationalEligibility      (component, space, space_2) float64 nan ......
    2d_capacityMax                (component, space, space_2) float64 nan ......
    2d_investPerCapacity          (component, space, space_2) float64 nan ......
    2d_investIfBuilt              (component, space, space_2) float64 nan ......
    2d_opexPerOperation           (component, space, space_2) float64 nan ......
    2d_opexPerCapacity            (component, space, space_2) float64 nan ......
    2d_opexIfBuilt                (component, space, space_2) float64 nan ......
    2d_QPcostScale                (component, space, space_2) float64 nan ......
    2d_interestRate               (component, space, space_2) float64 nan ......
    2d_economicLifetime           (component, space, space_2) float64 nan ......
    2d_technicalLifetime          (component, space, space_2) float64 nan ......
    2d_capacityFix                (component, space, space_2) float64 nan ......
    2d_reactances                 (component, space, space_2) float64 nan ......
    2d_losses                     (component, space, space_2) float64 nan ......
    2d_distances                  (component, space, space_2) float64 nan ......
    1d_locationalEligibility      (component, space) float64 1.0 1.0 ... nan nan
    1d_capacityMax                (component, space) float64 nan nan ... nan nan
    1d_investPerCapacity          (component, space) float64 0.65 0.65 ... nan
    1d_investIfBuilt              (component, space) float64 0.0 0.0 ... nan nan
    1d_opexPerOperation           (component, space) float64 0.0 0.0 ... nan nan
    1d_commodityCost              (component, space) float64 nan nan ... nan nan
    1d_commodityRevenue           (component, space) float64 nan nan ... nan nan
    1d_opexPerCapacity            (component, space) float64 0.021 0.021 ... nan
    1d_opexIfBuilt                (component, space) float64 0.0 0.0 ... nan nan
    1d_QPcostScale                (component, space) float64 0.0 0.0 ... nan nan
    1d_interestRate               (component, space) float64 0.08 0.08 ... nan
    1d_economicLifetime           (component, space) float64 33.0 33.0 ... nan
    1d_technicalLifetime          (component, space) float64 33.0 33.0 ... nan
    1d_capacityFix                (component, space) float64 nan nan ... nan nan
    1d_opexPerChargeOperation     (component, space) float64 nan nan ... nan nan
    1d_opexPerDischargeOperation  (component, space) float64 nan nan ... nan nan

## update_dicts_based_on_xarray_dataset()

In [ ]:
def update_dicts_based_on_xarray_dataset(esm_dict, component_dict, xarray_dataset):
    """Replaces dimensional data and respective descriptions in component_dict and esm_dict with spatially aggregated data 
    from xarray_dataset.

    Note:  Here, "dimensional data" refers to data containing at least one of the dimensions of time and space.

    :param esm_dict: dictionary containing information about the esM instance
    :type esm_dict: dict

    :param component_dict: dictionary containing information about the esM instance's components
    :type component_dict: dict

    :param xarray_dataset: dataset containing all "dimensional data" of an esM instance
    :type xarray_dataset: xarray.dataset

    :return: esm_dict, component_dict - updated dictionaries containing spatially aggregated data
    """
    
    df_iteration_dict, series_iteration_dict = generate_iteration_dicts(esm_dict, component_dict)

    # update esm_dict
    esm_dict['locations'] = set(str(value) for value in xarray_dataset.space.values)
    
    # update component_dict
    # set all regional time series (regions, time)
    for variable_description, description_tuple_list in df_iteration_dict.items():
        for description_tuple in description_tuple_list:
            classname, component_description = description_tuple

            df_description = f"{classname}, {component_description}"
            df = xarray_dataset[variable_description].sel(component=df_description).drop("component").to_dataframe().unstack(level=2)
            
            if len(df.columns) > 1:
                df.columns = df.columns.droplevel(0)

            component_dict[classname][component_description][variable_description] = df.sort_index()


    # set all 2d data (regions, regions)
    for variable_description, description_tuple_list in series_iteration_dict.items():

        for description_tuple in description_tuple_list:
            classname, component_description = description_tuple

            df_description = f"{classname}, {component_description}"

            if classname in ['Transmission', 'LinearOptimalPowerFlow']:
                series = xarray_dataset[f"2d_{variable_description}"].sel(component=df_description
                                                            ).drop("component").to_dataframe().stack(level=0)

                series.index = series.index.droplevel(level=2).map('_'.join)

                component_dict[classname][component_description][variable_description] = series.sort_index()


    # set all 1d data (regions)
    for variable_description, description_tuple_list in series_iteration_dict.items():

        for description_tuple in description_tuple_list:
            classname, component_description = description_tuple

            df_description = f"{classname}, {component_description}"

            if classname not in ['Transmission', 'LinearOptimalPowerFlow']:

                if variable_description not in ['commodityConversionFactors']: # TODO: this is a bugfix, properly implement this
                    # print(xarray_dataset[f"1d_{variable_description}"].sel(component=df_description))
                    series = xarray_dataset[f"1d_{variable_description}"].sel(component=df_description
                                                                            ).drop("component").to_dataframe().unstack(level=0)
                    series.index = series.index.droplevel(level=0)

                    component_dict[classname][component_description][variable_description] = series.sort_index()

    return esm_dict, component_dict

## spatial_aggregation()

In [ ]:
def spatial_aggregation(esM, numberOfRegions, gdfRegions=None, aggregation_function_dict=None, clusterMethod="centroid-based", 
                        aggregatedShapefileFolderPath=None, **kwargs):
    """Clusters the spatial data of all components considered in the EnergySystemModel instance and returns a new esM instance 
    with the aggregated data.        
    
    Additional keyword arguments for the SpatialAggregation instance can be added (facilitated by kwargs). 
    
    Please refer to the SPAGAT package documentation for more information.

    :param esM: energy system model instance 
        |br| * the default value is None
    :type esM: energySystemModelInstance

    :param numberOfRegions: states the number of regions into which the spatial data
        should be clustered.
        Note: Please refer to the SPAGAT package documentation of the parameter numberOfRegions for more
        information.
        |br| * the default value is None
    :type numberOfRegions: strictly positive integer, None

    **Default arguments:**

    :param gdfRegions: geodataframe containing the shapes of the regions of the energy system model instance
        |br| * the default value is None
    :type gdfRegions: geopandas.dataframe

    :param aggregatedShapefileFolderPath: indicate the path to the folder were the input and aggregated shapefiles shall be located 
        |br| * the default value is None
    :type aggregatedShapefileFolderPath: string

    :param clusterMethod: states the method which is used in the SPAGAT package for clustering the spatial
        data. Options are for example 'centroid-based'.
        Note: Please refer to the SPAGAT package documentation of the parameter clusterMethod for more information.
        |br| * the default value is 'centroid-based'
    :type clusterMethod: string

    :return: esM_aggregated - esM instance with spatially aggregated data and xarray dataset containing all spatially 
    resolved data
    """

    # initialize spagat_manager
    spagat_manager = spm.SpagatManager()
    esm_dict, component_dict = fn.dictIO.exportToDict(esM)
    spagat_manager.sds.xr_dataset = dimensional_data_to_xarray_dataset(esm_dict, component_dict)

    if gdfRegions is not None:
        spagat_manager.sds.add_objects(description='gpd_geometries',
                                       dimension_list=['space'],
                                       object_list=gdfRegions.geometry)
        spr.add_region_centroids(spagat_manager.sds, spatial_dim='space')

    # spatial clustering 
    spagat_manager.grouping(dimension_description='space')

    # representation of the clustered regions
    if aggregation_function_dict is not None:
        spagat_manager.aggregation_function_dict = aggregation_function_dict
    else:
        spagat_manager.aggregation_function_dict = {'operationRateMax': ('mean', None), # ('weighted mean', 'capacityMax')
                                                'operationRateFix': ('sum', None),
                                                'locationalEligibility': ('bool', None), 
                                                'capacityMax': ('sum', None),
                                                'investPerCapacity': ('sum', None), # ?
                                                'investIfBuilt': ('sum', None), # ? 
                                                'opexPerOperation': ('sum', None), # ?
                                                'opexPerCapacity': ('sum', None), # ?
                                                'opexIfBuilt': ('sum', None), # ?
                                                'interestRate': ('mean', None), # ?
                                                'economicLifetime': ('mean', None), # ?
                                                'capacityFix': ('sum', None),
                                                'losses': ('mean', None), # ?
                                                'distances': ('mean', None), # weighted mean ?
                                                'commodityCost': ('mean', None), # ?
                                                'commodityRevenue': ('mean', None), # ?
                                                'opexPerChargeOperation': ('mean', None),
                                                'opexPerDischargeOperation': ('mean', None),
                                            }

    spagat_manager.aggregation_function_dict = {f"{dimension}_{key}": value 
                                                for key, value in spagat_manager.aggregation_function_dict.items()
                                            for dimension in ["1d", "2d"]}

    spagat_manager.representation(number_of_regions=numberOfRegions)

    # create aggregated esM instance

    esmDict, compDict = update_dicts_based_on_xarray_dataset(esm_dict, component_dict, 
                                                             xarray_dataset=spagat_manager.sds_out.xr_dataset)
    
    esM_aggregated = fn.dictIO.importFromDict(esmDict, compDict)

    if aggregatedShapefileFolderPath is not None:

        # create region shape file
        aggregated_regions_FilePath = os.path.join(aggregatedShapefileFolderPath, 'aggregated_regions.shp')

        spagat_manager.sds_out.save_sds_regions(aggregated_regions_FilePath)

        # df_aggregated = spagat_manager.sds_out.xr_dataset.gpd_geometries.to_dataframe(
        #     ).reset_index(level=0).rename(columns={'space':'index', 'gpd_geometries': 'geometry'})

        # gdf_aggregated = gpd.GeoDataFrame(df_aggregated)
        # gdf_aggregated.crs = {'init' :'epsg:3035'}


        # create grid shape file
        aggregated_grid_FilePath = os.path.join(aggregatedShapefileFolderPath, 'aggregated_grid.shp')
        # spr.create_grid_shapefile(spagat_manager.sds_out, filename=aggregated_grid_FilePath)

    return esM_aggregated